## Identifying Parcels with No Physical Improvements (Cook County)

This notebook processes parcel-level data from Cook County to isolate PINs where **no improvements** have been made — such as the absence of buildings, construction, or renovations. These vacant or unimproved parcels are critical for identifying **underutilized land** for potential redevelopment, planning, or investment.

### Data Source:
- Dataset: `final_cook_county_data_with_same_dtype.csv`
- This file contains parcel identifiers along with structural details like:
  - `building_sqft`, `num_rooms`, `num_full_baths`, `garage_size`, `year_built`, etc.

### Process Overview:
1. Load parcel data in chunks to handle large size efficiently.
2. Define rules for detecting unimproved parcels — typically rows where key structural fields are null or zero.
3. Filter parcels that meet the “no improvement” criteria.
4. Deduplicate by PIN, keeping only the latest available `tax_year`.
5. Export a clean list of unimproved parcels for further use.

### ✅ Output:
- A cleaned CSV file containing **only unimproved parcels**, each represented by a unique PIN and its most recent tax record.
- This dataset can be merged with other layers (e.g., Connected Communities, transit access, city-owned land) for policy analysis and development planning.

In [2]:
import pandas as pd
from collections import defaultdict
import numpy as np

# Path to your large file
input_file = 'C:/Users/kaur6/Downloads/Urban Analytics/final_cook_county_data_with_same_dtype.csv'
chunk_size = 1000000
# Columns required for scoring (used inside is_improved), but we read all columns
scoring_columns = [
    'building_sqft',
    'year_built',
    'num_rooms',
    'num_bedrooms',
    'num_full_baths',
    'type_of_residence',
    'construction_quality'
]

vacant_rows = []
chunk_num = 0

def is_improved(row):
    signals = [
        not pd.isna(row['building_sqft']) and row['building_sqft'] >= 196,
        not pd.isna(row['year_built']) and row['year_built'] > 0,
        not pd.isna(row['num_rooms']) and row['num_rooms'] > 0,
        not pd.isna(row['num_bedrooms']) and row['num_bedrooms'] > 0,
        not pd.isna(row['num_full_baths']) and row['num_full_baths'] > 0,
        pd.notna(row['type_of_residence']),
        pd.notna(row['construction_quality'])
    ]
    return sum(signals) >= 3

for chunk in pd.read_csv(input_file, chunksize=chunk_size, low_memory=False):
    chunk_num += 1
    print(f"\n🔄 Processing chunk {chunk_num}...")

    # Filter using only scoring columns but keep all data
    vacant_chunk = chunk[~chunk[scoring_columns].apply(is_improved, axis=1)]

    vacant_rows.append(vacant_chunk)

    print(f"✅ Finished chunk {chunk_num} | Vacant this chunk: {len(vacant_chunk)}")

# Combine and save
vacant_df = pd.concat(vacant_rows, ignore_index=True)
vacant_df.to_csv('C:/Users/kaur6/Downloads/Urban Analytics/parcels_no_improvement_full_columns.csv', index=False)

print(f"\n🎯 Final: {len(vacant_df)} parcels classified as not clearly improved (with all original columns).")


🔄 Processing chunk 1...
✅ Finished chunk 1 | Vacant this chunk: 1342

🔄 Processing chunk 2...
✅ Finished chunk 2 | Vacant this chunk: 616

🔄 Processing chunk 3...
✅ Finished chunk 3 | Vacant this chunk: 387

🔄 Processing chunk 4...
✅ Finished chunk 4 | Vacant this chunk: 667

🔄 Processing chunk 5...
✅ Finished chunk 5 | Vacant this chunk: 344

🔄 Processing chunk 6...
✅ Finished chunk 6 | Vacant this chunk: 553

🔄 Processing chunk 7...
✅ Finished chunk 7 | Vacant this chunk: 748

🔄 Processing chunk 8...
✅ Finished chunk 8 | Vacant this chunk: 757

🔄 Processing chunk 9...
✅ Finished chunk 9 | Vacant this chunk: 1207

🔄 Processing chunk 10...
✅ Finished chunk 10 | Vacant this chunk: 1617

🔄 Processing chunk 11...
✅ Finished chunk 11 | Vacant this chunk: 3154

🔄 Processing chunk 12...
✅ Finished chunk 12 | Vacant this chunk: 5084

🔄 Processing chunk 13...
✅ Finished chunk 13 | Vacant this chunk: 7252

🔄 Processing chunk 14...
✅ Finished chunk 14 | Vacant this chunk: 1464

🔄 Processing chu

In [3]:
# Load the CSV with all columns
df = pd.read_csv('C:/Users/kaur6/Downloads/Urban Analytics/parcels_no_improvement_full_columns.csv')

# Ensure tax_year is numeric for proper comparison
df['tax_year'] = pd.to_numeric(df['tax_year'], errors='coerce')

# Drop rows with missing tax_year
df = df.dropna(subset=['tax_year'])

# Sort by pin and tax_year (latest first), then drop duplicates
df_sorted = df.sort_values(by=['pin', 'tax_year'], ascending=[True, False])
df_deduped = df_sorted.drop_duplicates(subset='pin', keep='first')

# Save the cleaned version
df_deduped.to_csv('C:/Users/kaur6/Downloads/Urban Analytics/parcels_no_improvement_unique_pin_latest_year.csv', index=False)

print(f"✅ Done! Final rows with unique pins and latest tax_year: {len(df_deduped)}")

C:\Users\kaur6\AppData\Local\Temp\ipykernel_5956\362984273.py:2: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/kaur6/Downloads/Urban Analytics/parcels_no_improvement_full_columns.csv')


✅ Done! Final rows with unique pins and latest tax_year: 18725
